In [1]:
import pandas as pd
import seaborn as sns
import networkx as nx
import numpy as np
import re
import matplotlib.pyplot as plt
import itertools
import sklearn.datasets
from sklearn.metrics.pairwise import euclidean_distances
import math
import random

In [2]:
def createRandomNetwork(n, p):
    #indices = list(range(0,n))
    edges = []
    for i in range(0, n):
        for j in range(i + 1, n):
            r = random.uniform(0,1)
            if r <= p: # Random chance
                # Now check for conditions
                #if j in edges.keys() might not be neccessary
                edges.append((i,j))
    return edges

In [3]:
n = 550
indices = list(range(0,n))
G1 = nx.Graph()
G1.add_nodes_from(indices)
edges = createRandomNetwork(n, 0.001)
G1.add_edges_from(edges)

In [4]:
edges

[(1, 469),
 (1, 499),
 (2, 137),
 (3, 422),
 (8, 110),
 (9, 281),
 (11, 464),
 (13, 432),
 (13, 464),
 (14, 522),
 (14, 542),
 (17, 57),
 (17, 223),
 (18, 131),
 (25, 46),
 (27, 319),
 (27, 338),
 (30, 334),
 (31, 507),
 (32, 131),
 (32, 156),
 (33, 171),
 (34, 132),
 (36, 38),
 (38, 291),
 (38, 457),
 (38, 461),
 (41, 442),
 (45, 225),
 (47, 424),
 (48, 107),
 (49, 229),
 (52, 397),
 (53, 89),
 (53, 309),
 (54, 133),
 (55, 136),
 (55, 205),
 (55, 503),
 (56, 469),
 (58, 243),
 (61, 447),
 (63, 102),
 (63, 344),
 (70, 336),
 (72, 509),
 (75, 352),
 (78, 241),
 (80, 181),
 (81, 299),
 (82, 138),
 (82, 325),
 (83, 341),
 (83, 496),
 (84, 99),
 (84, 239),
 (84, 242),
 (86, 493),
 (89, 422),
 (90, 392),
 (92, 392),
 (94, 283),
 (95, 335),
 (96, 448),
 (98, 524),
 (99, 304),
 (103, 437),
 (105, 116),
 (112, 128),
 (113, 415),
 (119, 494),
 (123, 161),
 (123, 367),
 (125, 132),
 (127, 148),
 (127, 465),
 (132, 545),
 (134, 521),
 (136, 493),
 (138, 143),
 (147, 196),
 (148, 365),
 (151, 256)

In [5]:
indices = list(range(0,n))
G2 = nx.Graph()
G2.add_nodes_from(indices)
edges = createRandomNetwork(n, 0.0059)
G2.add_edges_from(edges)

In [6]:
indices = list(range(0,n))
G3 = nx.Graph()
G3.add_nodes_from(indices)
edges = createRandomNetwork(n, 0.01)
G3.add_edges_from(edges)

In [7]:
df1 = nx.to_pandas_edgelist(G1)
df1.to_csv("Random1.csv")

In [8]:
df2 = nx.to_pandas_edgelist(G2)
df2.to_csv("Random2.csv")

In [9]:
df3 = nx.to_pandas_edgelist(G3)
df3.to_csv("Random3.csv")

In [10]:
nodes = list(range(0,n))
dfNodes = pd.DataFrame(data = nodes, columns=["Node"])
dfNodes.to_csv("RandomNodes.csv")

In [14]:
def save_graph_as_table(list_of_graphs):
    """
    Create a table with various graph properties for each graph in the list.
    """
    values = []
    for (G, p) in list_of_graphs:
        # Number of nodes and edges
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        
        # Average degree
        avg_degree = 2 * num_edges / num_nodes if num_nodes > 0 else 0
        
        # Density
        density = nx.density(G)
        
        # Connected components
        components = [len(c) for c in nx.connected_components(G)]
        num_components = len(components)
        largest_component_size = max(components) if components else 0
        
        # Diameter of the largest component (if applicable)
        if num_components > 1:
            largest_component = max(nx.connected_components(G), key=len)
            subgraph = G.subgraph(largest_component)
            diameter = nx.diameter(subgraph) if nx.is_connected(subgraph) else float('inf')
        else:
            diameter = nx.diameter(G) if nx.is_connected(G) else float('inf')
        
        # Average path length (computed per component)
        try:
            avg_path_length = nx.average_shortest_path_length(G)
        except nx.NetworkXError:
            avg_path_length = float('inf')
        
        # Clustering coefficient
        clustering_coefficient = nx.average_clustering(G)
        
        # Degree distribution (list of degrees of all nodes)
        degree_distribution = [d for _, d in G.degree()]
        
        # Community structure
        communities = list(nx.community.greedy_modularity_communities(G))
        modularity = nx.algorithms.community.modularity(G, communities)
        
        # Centralities
        degree_centrality = nx.degree_centrality(G)
        betweenness_centrality = nx.betweenness_centrality(G)
        
        # Adding all data to a dictionary for the table row
        values.append({
            "Vrcholy": num_nodes,
            "Hrany": num_edges,
            "Pravděpodobnost": p,
            "Průměrný stupeň": avg_degree,
            "Hustota": density,
            "Komponenty": num_components,
            "Velikosti komponentů": components,
            "Největší velikost komponentu": largest_component_size,
            "Průměr": diameter,
            "Průměrná délka cesty": avg_path_length,
            "Clustering Coefficient": clustering_coefficient,
            "Distribuce stupňů": degree_distribution,
            "Modularita": modularity,
            "Degree Centrality": degree_centrality,
            "Betweenness Centrality": betweenness_centrality
        })
    
    # Convert the list of dictionaries into a pandas DataFrame for tabular representation
    table = pd.DataFrame(values)
    display(table)
    
    table.to_csv("graph_metrics.csv", index=False)

In [15]:
save_graph_as_table([(G1, 0.001),(G2, 0.0059),(G3, 0.01)])

,Vrcholy,Hrany,Pravděpodobnost,Průměrný stupeň,Hustota,Komponenty,Velikosti komponentů,Největší velikost komponentu,Průměr,Průměrná délka cesty,Clustering Coefficient,Distribuce stupňů,Modularita,Degree Centrality,Betweenness Centrality
0,550,156,0.0010,0.567273,0.001033,394,"[1, 5, 2, 9, 1, 1, 1, 1, 2, 4, 1, 4, 1, 3, 1, ...",14,7,inf,0.000000,"[0, 2, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 2, 2, ...",0.971072,"{0: 0.0, 1: 0.0036429872495446266, 2: 0.001821...","{0: 0.0, 1: 2.6591147806895084e-05, 2: 0.0, 3:..."
1,550,874,0.0059,3.178182,0.005789,23,"[524, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 2...",524,12,inf,0.002121,"[2, 4, 4, 4, 4, 5, 1, 3, 5, 3, 5, 4, 4, 1, 4, ...",0.595290,"{0: 0.0036429872495446266, 1: 0.00728597449908...","{0: 0.0013823204451819183, 1: 0.01073234439175..."
2,550,1489,0.0100,5.414545,0.009863,4,"[547, 1, 1, 1]",547,8,inf,0.007695,"[1, 1, 1, 7, 9, 4, 7, 11, 5, 4, 8, 4, 5, 7, 8,...",0.417538,"{0: 0.0018214936247723133, 1: 0.00182149362477...","{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0065402540930098..."
